In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
df = pd.read_csv('../Data/csv/merge_homicidios_lesiones.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28315 entries, 0 to 28314
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Id_hechos             28315 non-null  object 
 1   fecha                 28315 non-null  object 
 2   hora                  28315 non-null  object 
 3   lugar_hecho           28262 non-null  object 
 4   tipo_calle            28315 non-null  object 
 5   altura                14884 non-null  float64
 6   comuna                28137 non-null  object 
 7   geocodificacion_CABA  28276 non-null  object 
 8   latitud               28045 non-null  object 
 9   longitud              28045 non-null  object 
 10  nro_victimas          28315 non-null  int64  
 11  participantes         27598 non-null  object 
 12  rol                   717 non-null    object 
 13  victima               27598 non-null  object 
 14  vehiculo_victima      28315 non-null  object 
 15  vehiculo_acusado   

transformamos tipos de datos

In [ ]:
# df['edad'] = df['edad'].replace(['SD', 'sd'], np.nan)
# df['edad'] = pd.to_numeric(df['edad'], errors='coerce')
# df['edad'] = df['edad'].astype('Int64')

# df['comuna'] = df['comuna'].replace(['SD', 'No Especificada'], np.nan)
# df['comuna'] = pd.to_numeric(df['comuna'], errors='coerce', downcast='integer').astype('Int64')

# df['fecha'] = pd.to_datetime(df['fecha'])

# df['fecha_fallecimiento'] = pd.to_datetime(df['fecha_fallecimiento'], errors='coerce')

# df['hora'] = pd.to_datetime(df['hora'], errors='coerce').dt.time

# df['sexo'] = df['sexo'].replace({'Varon': 'Masculino', 'Mujer': 'Femenino'})
# df = df[df['sexo'].isin(['Masculino', 'Femenino', 'Sd'])]

### Dataframe censo 2022 Caba

In [ ]:
df_poblacion_caba = pd.read_excel('../Data/excel/Censo_nacional_2022.xlsx')

In [ ]:
df_poblacion_caba.head()

,"Censo Nacional de Población, Hogares y Viviendas 2022",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,"Cuadro resumen. Total de población, población ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Jurisdicción,Total,NaN,NaN,NaN,Varones,NaN,NaN,NaN,Mujeres,NaN,NaN,NaN
2,NaN,Total de población,Población en viviendas particulares,Población en viviendas colectivas (¹),Población en situación de calle(²),Total de población,Población en viviendas particulares,Población en viviendas colectivas (¹),Población en situación de calle(²),Total de población,Población en viviendas particulares,Población en viviendas colectivas (¹),Población en situación de calle(²)
3,Total del país,45892285,45618787,267793,5705,22186791,22010881,171436,4474,23705494,23607906,96357,1231
4,Ciudad Autónoma de Buenos Aires,3121707,3095454,23850,2403,1439873,1428781,9216,1876,1681834,1666673,14634,527


In [ ]:
poblacion_total_caba = df_poblacion_caba['Unnamed: 1'][4]
poblacion_total_caba = pd.to_numeric(poblacion_total_caba, errors='coerce')
print(f"Población Total de CABA (Censo 2022): {poblacion_total_caba}")

Población Total de CABA (Censo 2022): 3121707


## KPI Reduccion en un 10% la tasa de homicidios en siniestros viales de los últimos seis meses

Tasa de Homicidios=( Numero de homicidios en siniestros viales / Poblacion total )×100,000

In [ ]:
fecha_fallecimiento = df[df['fecha_fallecimiento'].notnull()]

# Formula (Número de homicidios en siniestros viales / Población total) * 100,000
tasa_homicidios = (fecha_fallecimiento['nro_victimas'].sum() / poblacion_total_caba) * 100000
tasa_homicidios_ultimo_semestre = (fecha_fallecimiento['nro_victimas'].sum() / poblacion_total_caba) * 100000

reduccion = tasa_homicidios_ultimo_semestre * 0.9
print(f'Tasa de homicidios en siniestros viales del último semestre: {tasa_homicidios_ultimo_semestre:.2f} por 100,000 habitantes')

Tasa de homicidios en siniestros viales del último semestre: 22.07 por 100,000 habitantes


In [ ]:
resultados = {
    'Tasa de Homicidios Último Semestre': [tasa_homicidios_ultimo_semestre],
    'Objetivo de Reducción (10%)': [reduccion]
}

df_resultados = pd.DataFrame(resultados)
df_resultados

,Tasa de Homicidios Último Semestre,Objetivo de Reducción (10%)
0,22.071258,19.864132


In [ ]:
kpi_reduccion = pd.DataFrame({
    'Periodo': ['Último Semestre', 'Objetivo (10% Reducción)'],
    'Tasa de Homicidios': [tasa_homicidios_ultimo_semestre, reduccion]
})

fig = px.bar(kpi_reduccion, x='Periodo', y='Tasa de Homicidios', color='Periodo',
             labels={'Tasa de Homicidios': 'Tasa de Homicidios por 100,000 habitantes'},
             title='KPI Reduccion en un 10% la tasa de homicidios en siniestros viales de los últimos seis meses')

fig.show()

## KPI Reduccion en un 7% la cantidad de accidentes mortales de motociclistas en el último año

Cambio Porcentual =(Número de accidentes mortales con víctimas en moto en el año anterior - Número de accidentes mortales con víctimas en moto en el año actual) / (Número de accidentes mortales con víctimas en moto en el año anterior) * 100

In [ ]:
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')

accidentes_moto_por_anio = df[df['vehiculo_victima'] == 'Moto']
accidentes_moto_por_anio = accidentes_moto_por_anio.groupby(accidentes_moto_por_anio['fecha'].dt.year).size().reset_index(name='Cantidad')
accidentes_moto_por_anio.columns = ['Año', 'Cantidad']

accidentes_moto_por_anio['Porcentaje_Cambio'] = (
    (accidentes_moto_por_anio['Cantidad'].shift(1) - accidentes_moto_por_anio['Cantidad']) / 
    accidentes_moto_por_anio['Cantidad'].shift(1)
) * 100

print(accidentes_moto_por_anio)
porcentaje_reduccion_ultimo_anio = accidentes_moto_por_anio['Porcentaje_Cambio'].iloc[-1]

if porcentaje_reduccion_ultimo_anio <= -7:
    print("Se ha logrado una reducción del 7% o más en la cantidad de accidentes mortales de motociclistas en el último año.")
else:
    print("La reducción en la cantidad de accidentes mortales de moztociclistas en el último año no alcanza el 7%.")

    Año  Cantidad  Porcentaje_Cambio
0  2016        65                NaN
1  2017        56          13.846154
2  2018        57          -1.785714
3  2019      1199       -2003.508772
4  2020      2251         -87.739783
5  2021      2601         -15.548645
Se ha logrado una reducción del 7% o más en la cantidad de accidentes mortales de motociclistas en el último año.


El análisis del cambio porcentual entre años consecutivos, se observa que hubo aumentos significativos en el número de accidentes mortales de motociclistas en ciertos años, los datos actuales no cumplen con el objetivo del 7%, la combinación de esfuerzos para reducir la tasa de homicidios y mejorar la seguridad vial puede contribuir a un entorno más seguro en las carreteras.

En particular, el cambio porcentual extremadamente negativo entre 2018 y 2019 (-2003.51%) indica una disminución significativa en el número de accidentes mortales de motociclistas en 2019 en comparación con el año anterior. Sin embargo, este cambio porcentual negativo tan pronunciado podría deberse a una base de comparación muy baja en 2018.

Asimismo, entre 2019 y 2020, el cambio porcentual también es negativo (-87.74%), indicando una disminución sustancial en el número de accidentes mortales de motociclistas en 2020 en comparación con el año anterior. Este descenso podría estar relacionado con las restricciones y medidas implementadas durante la pandemia de COVID-19, que afectaron la movilidad y, por ende, la incidencia de accidentes de tráfico.

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=accidentes_moto_por_anio['Año'], y=accidentes_moto_por_anio['Cantidad'], mode='lines+markers', name='Cantidad de Accidentes'))
fig.add_trace(go.Scatter(x=accidentes_moto_por_anio['Año'], y=accidentes_moto_por_anio['Porcentaje_Cambio'], mode='lines+markers', name='Porcentaje de Cambio'))

fig.update_layout(title='KPI: Reducción de Accidentes Mortales de Motociclistas',
                  xaxis_title='Año',
                  yaxis_title='Cantidad / Porcentaje de Cambio',
                  legend=dict(x=0, y=1),
                  template='plotly_dark')
fig.show()